# Transfer learning: Densenet-161

## The code that produces the .csv file is at the very bottom.

In [1]:
import torch
import numpy as np

from torchvision.datasets import ImageFolder
from torchvision import transforms
import torch.nn as nn
from torch.optim import Adam, SGD
from torch.autograd import Variable
import PIL.Image as Image
from torchvision import models
import time
import os

In [2]:
normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    )

train_transform = transforms.Compose([
        #transforms.Resize(256),
        transforms.RandomRotation(45),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
        ])

valid_transform = transforms.Compose([
        #transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
            normalize,
        ])


train_dataset = ImageFolder(
        root='./bird_dataset/train_images/', 
        transform=train_transform
    )

valid_dataset = ImageFolder(
        root='./bird_dataset/val_images/', 
        transform=valid_transform
    )

In [3]:
batch_size = 16
num_workers = 1  #1 if cuda
pin_memory = True  #true if cuda
shuffle = True

train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, shuffle=shuffle,
        num_workers=num_workers, pin_memory=pin_memory,
    )

valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=batch_size, shuffle=shuffle,
        num_workers=num_workers, pin_memory=pin_memory,
    )

In [4]:
cuda_avail = torch.cuda.is_available()

model_ft = models.densenet161(pretrained=True)
num_ftrs = model_ft.classifier.in_features
model_ft.fc = nn.Linear(num_ftrs, 20)      

if cuda_avail:                                 
    model_ft = model_ft.cuda()   
    
loss_fn = nn.CrossEntropyLoss()          
optimizer = SGD(model_ft.parameters(), lr=0.0001, momentum=0.9)

/usr/local/lib/python3.5/dist-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#inspired from
# https://heartbeat.fritz.ai/basics-of-image-classification-with-pytorch-2f8973c51864


def save_models(epoch):
    torch.save(model_ft.state_dict(), "densenetmodel_{}.model".format(epoch))
    print("Checkpoint saved")

def test():
    model_ft.eval()
    test_acc = 0.0
    for data in valid_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model_ft(images)
        _,prediction = torch.max(outputs.data, 1)
        test_acc += (prediction == labels).sum().item()
        


  
    test_acc = test_acc / 103

    return test_acc

best_acc_train = 0.0
best_acc_test = 0.0

def train(num_epochs):
    global best_acc_train
    global best_acc_test
    for epoch in range(num_epochs):
        model_ft.train()
        train_acc = 0.0
        train_loss = 0.0
        for data in train_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
           
            optimizer.zero_grad()
            outputs = model_ft(images)
            loss = loss_fn(outputs,labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.cpu().item() * images.size(0)
            _, prediction = torch.max(outputs.data, 1)
            
            train_acc += (prediction == labels).sum().item()


        train_acc = train_acc / 1082
        train_loss = train_loss / 1082

        test_acc = test()

        if (test_acc >= best_acc_test) and (train_acc >= best_acc_train):
            save_models(epoch)
            best_acc_test = test_acc
            best_acc_train = train_acc


        # Print the metrics
        print("Epoch {}, Train Accuracy: {} , TrainLoss: {} , Test Accuracy: {}".format(epoch, train_acc, train_loss,test_acc))


In [6]:
optimizer = SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
train(20)

Checkpoint saved
Epoch 0, Train Accuracy: 0.28650646950092423 , TrainLoss: 3.9834477813318787 , Test Accuracy: 0.7475728155339806
Checkpoint saved
Epoch 1, Train Accuracy: 0.6719038817005545 , TrainLoss: 1.0638933970615296 , Test Accuracy: 0.8155339805825242
Checkpoint saved
Epoch 2, Train Accuracy: 0.7818853974121996 , TrainLoss: 0.7735497939388325 , Test Accuracy: 0.8446601941747572
Checkpoint saved
Epoch 3, Train Accuracy: 0.7846580406654344 , TrainLoss: 0.6915447671179851 , Test Accuracy: 0.8737864077669902
Epoch 4, Train Accuracy: 0.8031423290203327 , TrainLoss: 0.6565020024886634 , Test Accuracy: 0.8252427184466019
Epoch 5, Train Accuracy: 0.821626617375231 , TrainLoss: 0.583801226342672 , Test Accuracy: 0.8543689320388349
Epoch 6, Train Accuracy: 0.8345656192236599 , TrainLoss: 0.5492665238829946 , Test Accuracy: 0.8446601941747572
Epoch 7, Train Accuracy: 0.8512014787430684 , TrainLoss: 0.5411371585401722 , Test Accuracy: 0.8640776699029126
Epoch 8, Train Accuracy: 0.8456561922

In [10]:
model_ft.load_state_dict(torch.load('./resnetmodel_16.model'))
optimizer = SGD(model_ft.parameters(), lr=0.00001, momentum=0.1)
train(5)

Epoch 0, Train Accuracy: 0.9085027726432532 , TrainLoss: 0.2950058912173217 , Test Accuracy: 0.883495145631068
Epoch 1, Train Accuracy: 0.9094269870609981 , TrainLoss: 0.3191073614821637 , Test Accuracy: 0.9029126213592233
Epoch 2, Train Accuracy: 0.9038817005545287 , TrainLoss: 0.3278001830009348 , Test Accuracy: 0.9223300970873787
Epoch 3, Train Accuracy: 0.9177449168207024 , TrainLoss: 0.2944378697497567 , Test Accuracy: 0.8737864077669902
Epoch 4, Train Accuracy: 0.9075785582255084 , TrainLoss: 0.2885306401388262 , Test Accuracy: 0.8932038834951457


In [11]:
model_ft.load_state_dict(torch.load('./resnetmodel_16.model'))

In [12]:
test_dir = './bird_dataset/test_images/mistery_category'
output_file = open("testdensenet.csv", "w")
output_file.write("Id,Category\n")

def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')

for f in os.listdir(test_dir):
    if 'jpg' in f:
        data = valid_transform(pil_loader(test_dir + '/' + f))
        data = data.view(1, data.size(0), data.size(1), data.size(2))
        if cuda_avail:
            data = data.cuda()
        model_ft.eval()
        output = model_ft(data)
        pred = output.data.max(1, keepdim=True)[1]
        output_file.write("%s,%d\n" % (f[:-4], pred))

output_file.close()